#  Лаба 5. Прогнозирование оттока клиентов

In [1]:
import org.apache.spark.sql.{DataFrame,Dataset,Column,Row}
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.Pipeline
import sys.process._

## Читаем данные

In [82]:
"hdfs dfs -ls -h /labs/slaba05" !

Found 2 items
-rw-r--r--   3 hdfs hdfs     23.1 M 2022-01-06 18:46 /labs/slaba05/lab05_test.csv
-rw-r--r--   3 hdfs hdfs      168 M 2022-01-06 18:46 /labs/slaba05/lab05_train.csv


0

In [68]:
val train_df = spark.read
    .options(Map("inferSchema"->"true","header"->"true"))
    .csv("/labs/slaba05/lab05_train.csv")

val test_df = spark.read
    .options(Map("inferSchema"->"true","header"->"true"))
    .csv("/labs/slaba05/lab05_test.csv")

train_df = [_c0: int, ID: int ... 115 more fields]
test_df = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [69]:
train_df.show(10)

+------+------+--------------+-------------------+------------------------+--------------------+-----------------+----------------+----------------------+-----------------------+-----------------------+-------------------+---------------------+-----------------------+-------------------+--------------+-----------------+--------------------+-----------------------+--------------+------------------+--------------------+---------------+------------------+-----------------------+---------------------------+----------------------+------------------+--------------------+-------------------+-------------------+-------------+-----------------------+-----------------------+-----------------+---------------+-------------------+---------------+---------------+----------------+-------+---------------+-----------------------+-----------------+-----------+-------------------+-------------------+-------------+---+-----------------------+--------------+-----------------------+--------------------+----

In [70]:
train_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [74]:
train_df.rdd.getNumPartitions
// так изначально 2 партиции. Нормально.

2

## Разведочный анализ данных

In [86]:
// Сколько строк и столбцов в датафреймах
println(train_df.count, test_df.count)
println(train_df.columns.length, test_df.columns.length)

(320764,44399)
(117,116)


In [36]:
// статистика по полям . Смотрим во внешнем редакторе эту колбасу...
train_df.describe().show(10,20,false)

+-------+------------------+------------------+-------------------+--------------------+------------------------+--------------------+-----------------+----------------+----------------------+-----------------------+-----------------------+-------------------+---------------------+-----------------------+-------------------+--------------+-----------------+--------------------+-----------------------+--------------+------------------+--------------------+-------------------+------------------+-----------------------+---------------------------+----------------------+--------------------+--------------------+-------------------+-------------------+-------------+-----------------------+-----------------------+-----------------+---------------+-------------------+---------------+---------------+------------------+-------+--------------------+-----------------------+-----------------+------------------+-------------------+-------------------+--------------------+------------------+--------

In [87]:
// Функция подсчета количество null-ов во всех колонках датафрейма
def countNulls(columns:Array[String]):Array[Column]={
    columns.map(c=>{
      count(when(col(c).isNull,c)).alias(c)
    })
}

countNulls: (columns: Array[String])Array[org.apache.spark.sql.Column]


In [88]:
// Смотрим кол-во пропусков в данных
train_df.select(countNulls(train_df.columns):_*).show(200, 40, true)

-RECORD 0-----------------------------
 _c0                         | 0      
 ID                          | 0      
 CR_PROD_CNT_IL              | 0      
 AMOUNT_RUB_CLO_PRC          | 34550  
 PRC_ACCEPTS_A_EMAIL_LINK    | 180814 
 APP_REGISTR_RGN_CODE        | 265914 
 PRC_ACCEPTS_A_POS           | 180814 
 PRC_ACCEPTS_A_TK            | 180814 
 TURNOVER_DYNAMIC_IL_1M      | 1      
 CNT_TRAN_AUT_TENDENCY1M     | 251229 
 SUM_TRAN_AUT_TENDENCY1M     | 251229 
 AMOUNT_RUB_SUP_PRC          | 34551  
 PRC_ACCEPTS_A_AMOBILE       | 180814 
 SUM_TRAN_AUT_TENDENCY3M     | 220560 
 CLNT_TRUST_RELATION         | 257829 
 PRC_ACCEPTS_TK              | 180814 
 PRC_ACCEPTS_A_MTP           | 180814 
 REST_DYNAMIC_FDEP_1M        | 1      
 CNT_TRAN_AUT_TENDENCY3M     | 220560 
 CNT_ACCEPTS_TK              | 180814 
 APP_MARITAL_STATUS          | 258989 
 REST_DYNAMIC_SAVE_3M        | 1      
 CR_PROD_CNT_VCU             | 1      
 REST_AVG_CUR                | 1      
 CNT_TRAN_MED_TENDENCY1M 

In [89]:
// Посмотрим детальнее на поле TARGET. Есть один null ...
train_df.groupBy($"TARGET").count.orderBy($"count".desc).show()

+------+------+
|TARGET| count|
+------+------+
|     0|294607|
|     1| 26156|
|  null|     1|
+------+------+



In [91]:
// Пропусков очень много в полях (больше половины), исправлять их в лом, так что попробуем тупо выкинуть,
// может этого будет достаточно. И также строку где таргет не заполнен.

val train_df_clean = train_df.na.drop(Seq("TARGET")).select(
    "TARGET",
    "ID",
    "AGE",
    "CLNT_SETUP_TENOR", //Срок жизни клиента в банке
    "REST_AVG_CUR", //Средние остатки по текущим счетам
    "REST_AVG_PAYM", //Средние остатки по зарплатным счетам
    "TURNOVER_PAYM", //Средние обороты по зарплатным счетам
    "TURNOVER_CC", //Средние обороты по кредитным картам

    "CR_PROD_CNT_IL", //Кол-во открытых продуктов за отчетный период (по категориям продуктов)
    "CR_PROD_CNT_VCU",
    "CR_PROD_CNT_TOVR",
    "CR_PROD_CNT_PIL",
    "CR_PROD_CNT_CC",
    "CR_PROD_CNT_CCFP",

    "TURNOVER_DYNAMIC_IL_1M", //Тренд по среднемесячным оборотам за отчетный период (1 или 3 месяца)
    "TURNOVER_DYNAMIC_CUR_1M",
    "TURNOVER_DYNAMIC_PAYM_1M",
    "TURNOVER_DYNAMIC_CC_1M",
    "TURNOVER_DYNAMIC_IL_3M",
    "TURNOVER_DYNAMIC_CUR_3M",
    "TURNOVER_DYNAMIC_PAYM_3M",
    "TURNOVER_DYNAMIC_CC_3M",

    "REST_DYNAMIC_FDEP_1M", //Тренд среднемесяцных остатков по продуктам за отчетный период ( за 1 или 3 месяца)
    "REST_DYNAMIC_IL_1M",
    "REST_DYNAMIC_CUR_1M",
    "REST_DYNAMIC_PAYM_1M",
    "REST_DYNAMIC_CC_1M",
    "REST_DYNAMIC_SAVE_3M",
    "REST_DYNAMIC_FDEP_3M",
    "REST_DYNAMIC_PAYM_3M",
    "REST_DYNAMIC_IL_3M",
    "REST_DYNAMIC_CUR_3M",
    "REST_DYNAMIC_CC_3M",

    "LDEAL_GRACE_DAYS_PCT_MED" //Прочие продуктовые параметры за отчетный период (кредитным договорам)
)


train_df_clean = [TARGET: int, ID: int ... 32 more fields]


[TARGET: int, ID: int ... 32 more fields]

In [92]:
// Смотрим, что получили. Ок, null-ов нет нигде
train_df_clean.select(countCols(train_df_clean_cut.columns):_*).show(100,20,true)

-RECORD 0-----------------------
 TARGET                   | 0   
 ID                       | 0   
 AGE                      | 0   
 CLNT_SETUP_TENOR         | 0   
 REST_AVG_CUR             | 0   
 REST_AVG_PAYM            | 0   
 TURNOVER_PAYM            | 0   
 TURNOVER_CC              | 0   
 CR_PROD_CNT_IL           | 0   
 CR_PROD_CNT_VCU          | 0   
 CR_PROD_CNT_TOVR         | 0   
 CR_PROD_CNT_PIL          | 0   
 CR_PROD_CNT_CC           | 0   
 CR_PROD_CNT_CCFP         | 0   
 TURNOVER_DYNAMIC_IL_1M   | 0   
 TURNOVER_DYNAMIC_CUR_1M  | 0   
 TURNOVER_DYNAMIC_PAYM_1M | 0   
 TURNOVER_DYNAMIC_CC_1M   | 0   
 TURNOVER_DYNAMIC_IL_3M   | 0   
 TURNOVER_DYNAMIC_CUR_3M  | 0   
 TURNOVER_DYNAMIC_PAYM_3M | 0   
 TURNOVER_DYNAMIC_CC_3M   | 0   
 REST_DYNAMIC_FDEP_1M     | 0   
 REST_DYNAMIC_IL_1M       | 0   
 REST_DYNAMIC_CUR_1M      | 0   
 REST_DYNAMIC_PAYM_1M     | 0   
 REST_DYNAMIC_CC_1M       | 0   
 REST_DYNAMIC_SAVE_3M     | 0   
 REST_DYNAMIC_FDEP_3M     | 0   
 REST_DYNA

# Модель

In [94]:
// Колонки для фичей - берем все кроме TARGET" и "ID"
val cols = Array(
    "AGE",
    "CLNT_SETUP_TENOR", //Срок жизни клиента в банке
    "REST_AVG_CUR", //Средние остатки по текущим счетам
    "REST_AVG_PAYM", //Средние остатки по зарплатным счетам
    "TURNOVER_PAYM", //Средние обороты по зарплатным счетам
    "TURNOVER_CC", //Средние обороты по кредитным картам

    "CR_PROD_CNT_IL", //Кол-во открытых продуктов за отчетный период (по категориям продуктов)
    "CR_PROD_CNT_VCU",
    "CR_PROD_CNT_TOVR",
    "CR_PROD_CNT_PIL",
    "CR_PROD_CNT_CC",
    "CR_PROD_CNT_CCFP",

    "TURNOVER_DYNAMIC_IL_1M", //Тренд по среднемесячным оборотам за отчетный период (1 или 3 месяца)
    "TURNOVER_DYNAMIC_CUR_1M",
    "TURNOVER_DYNAMIC_PAYM_1M",
    "TURNOVER_DYNAMIC_CC_1M",
    "TURNOVER_DYNAMIC_IL_3M",
    "TURNOVER_DYNAMIC_CUR_3M",
    "TURNOVER_DYNAMIC_PAYM_3M",
    "TURNOVER_DYNAMIC_CC_3M",

    "REST_DYNAMIC_FDEP_1M", //Тренд среднемесяцных остатков по продуктам за отчетный период ( за 1 или 3 месяца)
    "REST_DYNAMIC_IL_1M",
    "REST_DYNAMIC_CUR_1M",
    "REST_DYNAMIC_PAYM_1M",
    "REST_DYNAMIC_CC_1M",
    "REST_DYNAMIC_SAVE_3M",
    "REST_DYNAMIC_FDEP_3M",
    "REST_DYNAMIC_PAYM_3M",
    "REST_DYNAMIC_IL_3M",
    "REST_DYNAMIC_CUR_3M",
    "REST_DYNAMIC_CC_3M",

    "LDEAL_GRACE_DAYS_PCT_MED" //Прочие продуктовые параметры за отчетный период (кредитным договорам)
)

// Вектор с фичами
val assembler = new VectorAssembler()
  .setInputCols(cols)
  .setOutputCol("features")

// Используем алгоритм Gradient-boosted tree regression
val gbt = new GBTRegressor()
  .setLabelCol("TARGET")
  .setFeaturesCol("features")
  .setMaxIter(10)

// Валидация на ROC AUC score
val evaluator = new BinaryClassificationEvaluator()
  .setLabelCol("TARGET")
  .setRawPredictionCol("prediction")
  .setMetricName("areaUnderROC")

val pipeline = new Pipeline()
  .setStages(Array(assembler, gbt))

cols = Array(AGE, CLNT_SETUP_TENOR, REST_AVG_CUR, REST_AVG_PAYM, TURNOVER_PAYM, TURNOVER_CC, CR_PROD_CNT_IL, CR_PROD_CNT_VCU, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, CR_PROD_CNT_CC, CR_PROD_CNT_CCFP, TURNOVER_DYNAMIC_IL_1M, TURNOVER_DYNAMIC_CUR_1M, TURNOVER_DYNAMIC_PAYM_1M, TURNOVER_DYNAMIC_CC_1M, TURNOVER_DYNAMIC_IL_3M, TURNOVER_DYNAMIC_CUR_3M, TURNOVER_DYNAMIC_PAYM_3M, TURNOVER_DYNAMIC_CC_3M, REST_DYNAMIC_FDEP_1M, REST_DYNAMIC_IL_1M, REST_DYNAMIC_CUR_1M, REST_DYNAMIC_PAYM_1M, REST_DYNAMIC_CC_1M, REST_DYNAMIC_SAVE_3M, REST_DYNAMIC_FDEP_3M, REST_DYNAMIC_PAYM_3M, REST_DYNAMIC_IL_3M, REST_DYNAMIC_CUR_3M, REST_DYNAMIC_CC_3M, LDEAL_GRACE_DAYS_PCT_MED)
assembler = vecAssembler_46db6c72c55e


gbt: org.apache.spark.ml.regression.GBTRegressor...


vecAssembler_46db6c72c55e

In [96]:
// Обучение и валидация модели. 

// случайное разбиение на обучающую и проверочную части
val Array(train_data, val_data) = train_df_clean_cut.randomSplit(Array(0.65, 0.35))  
val model = pipeline.fit(train_data)
val predictions = model.transform(val_data)
val accuracy = evaluator.evaluate(predictions)
// accuracy > 0.80 - норм

train_data = [TARGET: int, ID: int ... 32 more fields]
val_data = [TARGET: int, ID: int ... 32 more fields]
model = pipeline_d5607926aacc
predictions = [TARGET: int, ID: int ... 34 more fields]
accuracy = 0.8033801345328687


0.8033801345328687

In [97]:
// Обучаем модель на всем трейне
val model = pipeline.fit(train_df_clean)
// Получаем предикты на тестовых данных
val predictions = model.transform(test_df)

model = pipeline_d5607926aacc
predictions = [_c0: int, ID: int ... 116 more fields]


[_c0: int, ID: int ... 116 more fields]

In [98]:
// Берем нужные колонки, переименовываем, смотрим что получили
val predictions_df = predictions
    .select("ID","prediction")
    .withColumnRenamed("ID","id")
    .withColumnRenamed("prediction","target")
pred_df.show()

+------+--------------------+
|    id|              target|
+------+--------------------+
|519130| 0.10532352712065535|
|234045|0.009365383704254266|
|401256|0.004809516691642...|
|551070|0.030328511215895974|
|367285|0.008593258670558546|
|497998|0.013571868107250017|
|413082|  0.1152834086062017|
|349893| -3.1037950042329E-4|
|346337| 0.13029977308402804|
|289979|0.026000123094179504|
|510818| 0.25701450311972346|
|235935|0.013917510022612992|
|532135| 0.33517723788538045|
|564760|0.014840315340617414|
|277391| 0.08362352397578383|
|336830| 0.32687509294332917|
|356053|0.010400102297599597|
|293302| 0.04010306298802175|
|322368|0.015456357395547807|
|406041|0.004650432073084594|
+------+--------------------+
only showing top 20 rows



pred_df = [id: int, target: double]


[id: int, target: double]

## Сохраняем результат

In [100]:
// Репартицируем в 1 партишен и сохраняем csv на HDFS
predictions_df.coalesce(1).write
    .option("header",true)
    .option("delimiter","\t")
    .csv("lab05.csv")

lastException: Throwable = null


In [104]:
// смотрим имя файла
"hdfs dfs -ls -h /user/dmitriy.kravtsov/lab05.csv" !

Found 2 items
-rw-r--r--   3 dmitriy.kravtsov dmitriy.kravtsov          0 2022-11-09 23:34 /user/dmitriy.kravtsov/lab05.csv/_SUCCESS
-rw-r--r--   3 dmitriy.kravtsov dmitriy.kravtsov      1.2 M 2022-11-09 23:34 /user/dmitriy.kravtsov/lab05.csv/part-00000-5d8da05a-f974-48c5-8e69-8b4efc24c74e-c000.csv


0

In [103]:
// копируем с переименованием полученный файл на локальную ФС
"hdfs dfs -get /user/dmitriy.kravtsov/lab05.csv/part-00000-5d8da05a-f974-48c5-8e69-8b4efc24c74e-c000.csv lab05.csv" !

0

In [ ]:
spark.stop()